# 🧠 Logger — Industry-Level Understanding (Locked In)

---

## 1️⃣ First: what a logger actually is (simple)

Think of a **logger like a news reporter**.

When something happens in your program:

- “Pipeline started” → reporter writes it down  
- “Error happened” → reporter writes it down  
- “Step finished” → reporter writes it down  

### Where does the reporter write?
- To the terminal  
- To a file  

That’s it.

---

## 2️⃣ Why we don’t use `print()`

### `print()`:
- Only shows on screen  
- Disappears after program ends  
- Not saved  
- Not structured  

### Logger:
- Saves history  
- Works in CI/CD  
- Has levels (`info`, `warning`, `error`)  

---

## 3️⃣ Main parts of logging (VERY IMPORTANT)

There are **4 core concepts**:

| Part | Meaning |
|-----|--------|
| Logger | The reporter |
| Handler | Where to write |
| Formatter | How to write |
| Level | Importance |

We’ll map each to code.

---

## 4️⃣ The logger code — explained LINE BY LINE

### 🔹 Step 1: imports

```python
import logging
import os
from datetime import datetime
from logging.handlers import RotatingFileHandler
```

- `logging` → Python’s logging system  
- `RotatingFileHandler` → prevents logs from growing forever  

---

### 🔹 Step 2: create logs folder

```python
LOG_DIR = os.path.join(os.getcwd(), "logs")
os.makedirs(LOG_DIR, exist_ok=True)
```

Meaning:

> “Create a folder named `logs`.  
> If it already exists, don’t crash.”

---

### 🔹 Step 3: log file name

```python
LOG_FILE_NAME = f"{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}.log"
LOG_FILE_PATH = os.path.join(LOG_DIR, LOG_FILE_NAME)
```

Meaning:

> “Each run gets its own timestamped log file.”

Example:

```
logs/2026_01_27_14_30_01.log
```

---

### 🔹 Step 4: log format

```python
LOG_FORMAT = "[%(asctime)s] [%(levelname)s] [%(name)s:%(lineno)d] %(message)s"
```

This controls **how each log line looks**.

Example output:

```
[2026-01-27 14:30:01] [ERROR] [data_ingestion:45] File not found
```

| Part | Meaning |
|-----|--------|
| asctime | Time |
| levelname | INFO / WARNING / ERROR |
| name | File name |
| lineno | Line number |
| message | Your log message |

---

## 5️⃣ The `get_logger()` function (CORE PART)

```python
def get_logger(name: str) -> logging.Logger:
```

Meaning:

> “Give me a logger for this file.”

You pass:

```python
__name__
```

So Python knows **which file is logging**.

---

### 🔹 Step 5.1: create the logger

```python
logger = logging.getLogger(name)
logger.setLevel(logging.INFO)
```

Meaning:

> “Create a logger and log INFO and above.”

- `INFO` → info, warning, error  
- `DEBUG` → everything  

---

### 🔹 Step 5.2: avoid duplicate logs (IMPORTANT)

```python
if logger.handlers:
    return logger
```

Without this:
- Logs appear twice  
- Very common beginner bug  

This says:

> “If logger already exists, reuse it.”

---

## 6️⃣ Handlers = WHERE logs go

### 🔹 File handler

```python
file_handler = RotatingFileHandler(
    LOG_FILE_PATH,
    maxBytes=5 * 1024 * 1024,
    backupCount=5
)
```

Meaning:

- Write logs to file  
- Max size = 5MB  
- Keep last 5 files  

Prevents **disk overflow**.

---

### 🔹 Console handler

```python
console_handler = logging.StreamHandler()
```

Meaning:

> “Also show logs in terminal.”

Critical for:
- Debugging  
- CI/CD pipelines  

---

## 7️⃣ Formatter = HOW logs look

```python
formatter = logging.Formatter(LOG_FORMAT)
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)
```

Same format for:
- File  
- Terminal  

---

## 8️⃣ Attach handlers to logger

```python
logger.addHandler(file_handler)
logger.addHandler(console_handler)
```

Now the logger knows:

✔ Write to file  
✔ Write to terminal  

---

## 9️⃣ How YOU use this logger in code

```python
logger = get_logger(__name__)

logger.info("Pipeline started")
logger.warning("Missing values detected")
logger.error("Data ingestion failed", exc_info=True)
```

---

## 🔥 VERY IMPORTANT: `exc_info=True`

```python
logger.error("Something failed", exc_info=True)
```

This prints:
- Error message  
- Full stack trace  

Without this:
❌ You won’t know **where it failed**

---

## 🧠 Mental model (LOCK THIS IN)

```
Logger     → who reports
Handler    → where to write
Formatter  → how it looks
Level      → importance
```

---

## 🎯 One-line explanation (INTERVIEW-READY)

> “We use a centralized logger with file and console handlers, structured formatting, and log rotation to ensure observability in both local runs and CI/CD pipelines.”

---

## ✅ Final reassurance

✔ You are NOT behind  
✔ Logging is confusing initially  
✔ You now understand the moving parts  
✔ This is **industry-level knowledge**


# ✅ INDUSTRY-STANDARD VERSION (recommended)

This is production-ready, clean, and widely used.

In [ ]:
import logging
import os
from datetime import datetime
from logging.handlers import RotatingFileHandler

LOG_DIR = os.path.join(os.getcwd(), "logs")
os.makedirs(LOG_DIR, exist_ok=True)

LOG_FILE_NAME = f"{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}.log"
LOG_FILE_PATH = os.path.join(LOG_DIR, LOG_FILE_NAME)

LOG_FORMAT = "[%(asctime)s] [%(levelname)s] [%(name)s:%(lineno)d] %(message)s"

def get_logger(name: str) -> logging.Logger:
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)

    if logger.handlers:
        return logger  # prevent duplicate logs

    formatter = logging.Formatter(LOG_FORMAT)

    # File handler (with rotation)
    file_handler = RotatingFileHandler(
        LOG_FILE_PATH,
        maxBytes=5 * 1024 * 1024,  # 5 MB
        backupCount=5
    )
    file_handler.setFormatter(formatter)

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger


# ✅ How you should use it (correct pattern)

In [ ]:
from src.common.logger import get_logger

logger = get_logger(__name__)

logger.info("Data ingestion started")

try:
    ...
except Exception as e:
    logger.error("Data ingestion failed", exc_info=True)
    raise
